# Explore NYC Street Flooding Complaints with MapPLUTO

## Import Libraries

### External Libraries

In [ ]:
import geopandas as gpd
import pandas as pd
import fiona

## Define Variables

### NYC Street Flooding Complaints

In [ ]:
nyc_street_flooding_geojson = 'data/street-flooding/street-flood-complaints_rows-all.geojson'

### MapPLUTO

In [ ]:
map_pluto_gdb_folder = 'data/PLUTO/MapPLUTO_22v3_1_water_included.gdb'

In [ ]:
map_pluto_shp_folder  = 'data/PLUTO/MapPLUTO_22v3_1_water_included.shp'

## Combine `Street Flooding Complaints` and `MapPLUTO` Datasets

### Read `Street Flooding Complaints`

In [ ]:
street_flooding_gdf = gpd.read_file(nyc_street_flooding_geojson)

EPSG: 4326

In [ ]:
len(street_flooding_gdf)

In [ ]:
street_flooding_gdf.crs

### Read `MapPluto`

In [ ]:
pluto_gdf = gpd.read_file(map_pluto_gdb_folder)

In [ ]:
pluto_shp_gdf = gpd.read_file(map_pluto_shp_folder)

In [ ]:
map_pluto_gdf = pluto_gdf.to_crs(4326)

In [ ]:
map_pluto_gdf.crs

In [ ]:
pluto_shp_gdf.crs

In [ ]:
map_pluto_gdf.columns

In [ ]:
map_pluto_gdb = fiona.open(map_pluto_gdb_folder)

EPSG: 2263

In [ ]:
map_pluto_gdb.crs

### Spatial Join

In [ ]:
street_flooding_map_pluto_df = (
    gpd.sjoin_nearest(
        street_flooding_gdf.to_crs(4326),
        map_pluto_gdf.to_crs(4326),
        distance_col = 'distance_between'
    ).reset_index(drop = True)
)

In [ ]:
street_flooding_map_pluto_shp_df = (
    gpd.sjoin(
        street_flooding_gdf,
        pluto_shp_gdf.to_crs(4326),
        how = 'inner',
        predicate = 'within'
    ).reset_index(drop = True)
)


In [ ]:
len(street_flooding_map_pluto_df)

In [ ]:
len(street_flooding_map_pluto_shp_df)

In [ ]:
street_flooding_map_pluto_shp_df.crs

In [ ]:
pluto_shp_gdf.crs

In [ ]:
street_flooding_map_pluto_shp_df['bbl']

In [ ]:
street_flooding_map_pluto_shp_df['BBL'] = street_flooding_map_pluto_shp_df['bbl'].astype(float)

In [ ]:
street_flooding_gdf

In [ ]:
street_flooding_map_pluto_shp_df['BBL']

In [ ]:
pluto_shp_gdf['BBL']

In [ ]:
street_flooding_gdf2 = street_flooding_gdf.copy()

In [ ]:
street_flooding_gdf2['BBL'] = street_flooding_gdf2['bbl'].astype(float)

In [ ]:
len(street_flooding_gdf2)

In [ ]:
street_flood_pluto_gdf = pd.merge(
    street_flooding_gdf2,
    pluto_shp_gdf.to_crs(4326),
    # left_on = 'bbl',
    # right_on = 'BBL',
    on = 'BBL',
    how = 'left'
)

In [ ]:
len(street_flood_pluto_gdf)

In [ ]:
street_flood_pluto_gdf.head()

In [ ]:
street_flood_pluto_gdf.columns

In [ ]:
nan_count = street_flood_pluto_gdf['geometry_y'].isna().sum()

In [ ]:
print(f'geometry available: {len(street_flood_pluto_gdf) - nan_count}\nnan: {nan_count}\npercent: {(len(street_flood_pluto_gdf) - nan_count)/len(street_flood_pluto_gdf)*100}%')

In [ ]:
street_flooding_map_pluto_df.info()

In [ ]:
street_flooding_map_pluto_df.head()